In [1]:
import pandas as pd

import torch
import argparse
import numpy as np
from datasets import Dataset
from evaluate import load
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

/mmfs1/gscratch/ark/knylund/.conda/envs/arkenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mmfs1/gscratch/ark/knylund/.conda/envs/arkenv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/mmfs1/gscratch/ark/knylund/.conda/envs/arkenv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [2]:
prefix = f"translate English to German: "
def preprocess(sample, tokenizer):
    inputs = prefix + str(sample["0"])
    targets = str(sample["1"])
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    labels = tokenizer(text_target=targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [26]:
#model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small").to("cuda")
model_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
ende_tokenizer = AutoTokenizer.from_pretrained("../mt_tokenizers/mt5-small_English-German_tokenizer")
enru_tokenizer = AutoTokenizer.from_pretrained("../mt_tokenizers/mt5-small_English-Russian_tokenizer")
enfi_tokenizer = AutoTokenizer.from_pretrained("../mt_tokenizers/mt5-small_English-Finnish_tokenizer")

ende_mtokenizer = AutoTokenizer.from_pretrained("../mt_tokenizers/gpt2_English-German_morpheme_tokenizer")
enru_mtokenizer = AutoTokenizer.from_pretrained("../mt_tokenizers/gpt2_English-Russian_morpheme_tokenizer")
enfi_mtokenizer = AutoTokenizer.from_pretrained("../mt_tokenizers/gpt2_English-Finnish_morpheme_tokenizer")
#if args.tokenizer != args.model:
model_tokenizer_vocab = model_tokenizer.get_vocab()
ende_tokenizer_vocab = ende_tokenizer.get_vocab()
enru_tokenizer_vocab = enru_tokenizer.get_vocab()
enfi_tokenizer_vocab = enfi_tokenizer.get_vocab()

ende_mtokenizer_vocab = ende_mtokenizer.get_vocab()
enru_mtokenizer_vocab = enru_mtokenizer.get_vocab()
enfi_mtokenizer_vocab = enfi_mtokenizer.get_vocab()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
de_path = "/mmfs1/gscratch/ark/knylund/bad-pair-encoding/paracrawl_data/German/dev-small"
de_pd = pd.read_csv(de_path, sep="\t", header=None, on_bad_lines="skip", engine="python")
de_dataset = Dataset.from_pandas(de_pd)

ru_path = "/mmfs1/gscratch/ark/knylund/bad-pair-encoding/paracrawl_data/Russian/dev-small"
ru_pd = pd.read_csv(ru_path, sep="\t", header=None, on_bad_lines="skip", engine="python")
ru_dataset = Dataset.from_pandas(ru_pd)

fi_path = "/mmfs1/gscratch/ark/knylund/bad-pair-encoding/paracrawl_data/Finnish/dev-small"
fi_pd = pd.read_csv(fi_path, sep="\t", header=None, on_bad_lines="skip", engine="python")
fi_dataset = Dataset.from_pandas(fi_pd)

In [142]:
print(de_pd)
print(fi_dataset)

                                                       0  \
0      Michael Ruse (2014), Philosophy of Science 81(...   
1            Category: Category Member Since: 2009-10-19   
2         Find your way through the world and to Gerard.   
3      Nawang hasn’t written anything in his profile ...   
4      Product details Tomatin three bridges - Ask a ...   
...                                                  ...   
24240  Wilde & Partner offers customized services to ...   
24241  It is perfect for indoor and outdoor wear. You...   
24242  Step 2: Install and launch 5K Player free YouT...   
24243  I can’t really do much about it, but it helps ...   
24244  The 10 best mechanical engineering teachers in...   

                                                       1  
0      Zeitschrift für philosophische Forschung 69 (2...  
1           Erobere den ersten Mitglied seit: 2012-03-04  
2      Durchlaufe die Welt und finde deinen Weg zum G...  
3       Nisha hat noch nichts in her Profil

In [6]:
print(len(set(model_tokenizer_vocab.keys()).difference(set(ende_tokenizer_vocab.keys()))))
print(len(set(model_tokenizer_vocab.keys()).difference(set(enru_tokenizer_vocab.keys()))))
print(len(set(model_tokenizer_vocab.keys()).difference(set(enfi_tokenizer_vocab.keys()))))

print(len(set(enru_mtokenizer_vocab.keys()).difference(set(enru_tokenizer_vocab.keys()))))
print(len(set(enfi_mtokenizer_vocab.keys()).difference(set(enfi_tokenizer_vocab.keys()))))

193289
189749
198776
226116
211413


In [28]:
example_num = 115

print("Sample: " + de_dataset[example_num]["1"])
#print(f'Default tokenization: {model_tokenizer.tokenize(dev_dataset[example_num]["1"])}')
print(model_tokenizer.tokenize(de_dataset[example_num]["1"]))
de_tokens = ende_tokenizer.tokenize(de_dataset[example_num]["1"])
#print(f"German-specific tokenization: {de_tokens}")
#print(de_tokens)

de_mtokens = ende_mtokenizer.tokenize(de_dataset[example_num]["1"])
de_mtokens = [mtok.replace("Ġ", '▁').replace("Ã¤", "ä").replace("âĢľ", "“") for mtok in de_mtokens]
print(de_mtokens)

re_tokenized = []
for i in range(len(de_tokens)):
    if de_tokens[i] not in model_tokenizer_vocab:
        def_de_tok = model_tokenizer.tokenize(de_tokens[i])
        #print(def_de_tok, de_tokens[i])
        if de_tokens[i][0] != '▁' and def_de_tok[0] == '▁':
            def_de_tok = def_de_tok[1:]
        elif def_de_tok[0][0] == '▁' and re_tokenized[-1] == '▁':
            del re_tokenized[-1]
        re_tokenized += def_de_tok
    else:
        re_tokenized.append(de_tokens[i])

print(re_tokenized)
#print(f"German and then default tokenized: {re_tokenized}")
#print(preprocess(dev_dataset[example_num], model_tokenizer))
#print(preprocess(dev_dataset[example_num], ende_tokenizer))

Sample: Kommt wieder, wann immer ihr wollt.“ Wow, wir waren wirklich sehr stolz auf dieses Feedback.
['▁Komm', 't', '▁wieder', ',', '▁', 'wann', '▁immer', '▁', 'ihr', '▁woll', 't', '.“', '▁Wow', ',', '▁wir', '▁', 'waren', '▁w', 'irklich', '▁', 'sehr', '▁stol', 'z', '▁auf', '▁diese', 's', '▁Feedback', '.']
['Kommt', '▁wieder', ',', '▁wann', '▁immer', '▁ihr', '▁wollt', '.“', '▁Wow', ',', '▁wir', '▁waren', '▁wirklich', '▁sehr', '▁stolz', '▁auf', '▁dieses', '▁Feedback', '.']
['▁Komm', 't', '▁wieder', ',', '▁', 'wann', '▁immer', '▁', 'ihr', '▁woll', 't', '.“', '▁Wow', ',', '▁wir', '▁', 'waren', '▁w', 'irklich', '▁', 'sehr', '▁stol', 'z', '▁auf', '▁diese', 's', '▁Feedback', '.']


In [16]:
example_num = 2300

print("Sample: " + fi_dataset[example_num]["1"])
#print(f'Default tokenization: {model_tokenizer.tokenize(dev_dataset[example_num]["1"])}')
print(model_tokenizer.tokenize(fi_dataset[example_num]["1"]))
fi_tokens = enfi_tokenizer.tokenize(fi_dataset[example_num]["1"])
print(fi_tokens)

fi_mtokens = enfi_mtokenizer.tokenize(fi_dataset[example_num]["1"])
fi_mtokens = [mtok.replace("Ġ", '▁') for mtok in fi_mtokens]
print(fi_mtokens)

re_tokenized = []
for i in range(len(fi_mtokens)):
    if fi_mtokens[i] not in model_tokenizer_vocab:
        def_fi_tok = model_tokenizer.tokenize(fi_mtokens[i], add_special_tokens=False)
        #print(def_de_tok, de_tokens[i])
        if len(def_fi_tok) > 0:
            if fi_mtokens[i][0] != '▁' and def_fi_tok[0] == '▁':
                def_fi_tok = def_fi_tok[1:]
            elif def_fi_tok[0][0] == '▁' and len(re_tokenized) > 0 and re_tokenized[-1] == '▁':
                del re_tokenized[-1]
            re_tokenized += def_fi_tok
    else:
        re_tokenized.append(fi_mtokens[i])

print(re_tokenized)
#print(preprocess(fi_dataset[example_num], model_tokenizer)["labels"])
#print(preprocess(fi_dataset[example_num], enfi_tokenizer)["labels"])
#print(preprocess(fi_dataset[example_num], enfi_mtokenizer)["labels"])
#print(model_tokenizer.convert_tokens_to_ids(re_tokenized) + [1])

Sample: Joulunodotus käynnistyy Vanhan Suurtorin Joulumarkkinoilta
['▁Joulu', 'no', 'dotus', '▁', 'käynnisty', 'y', '▁Van', 'han', '▁Suur', 'tor', 'in', '▁Joulu', 'markkinoi', 'lta']
['▁Joulun', 'odotus', '▁käynnisty', 'y', '▁Vanhan', '▁Suur', 'torin', '▁Joulu', 'markkinoilta']
['Joulun', 'odotus', '▁käynnistyy', '▁Vanhan', '▁Su', 'urt', 'orin', '▁Joul', 'umarkkin', 'oilta']
['▁Joulu', 'n', '▁odot', 'us', '▁', 'käynnisty', 'y', '▁Van', 'han', '▁Su', 'ur', 't', 'orin', '▁Jo', 'ul', 'u', 'mark', 'kin', '▁oil', 'ta']


In [24]:
example_num = 10

print("Sample: " + ru_dataset[example_num]["1"])
#print(f'Default tokenization: {model_tokenizer.tokenize(dev_dataset[example_num]["1"])}')
print(model_tokenizer.tokenize(ru_dataset[example_num]["1"]))
fi_tokens = enru_tokenizer.tokenize(ru_dataset[example_num]["1"])
print(fi_tokens)

fi_mtokens = enru_mtokenizer.tokenize(ru_dataset[example_num]["1"])
fi_mtokens = [mtok.replace("Ġ", '▁').replace("Ã¤", "ä") for mtok in fi_mtokens]
print(fi_mtokens)

re_tokenized = []
for i in range(len(fi_tokens)):
    if fi_tokens[i] not in model_tokenizer_vocab:
        def_fi_tok = model_tokenizer.tokenize(fi_tokens[i], add_special_tokens=False)
        #print(def_de_tok, de_tokens[i])
        if len(def_fi_tok) > 0:
            if fi_tokens[i][0] != '▁' and def_fi_tok[0] == '▁':
                def_fi_tok = def_fi_tok[1:]
            elif def_fi_tok[0][0] == '▁' and len(re_tokenized) > 0 and re_tokenized[-1] == '▁':
                del re_tokenized[-1]
            re_tokenized += def_fi_tok
    else:
        re_tokenized.append(fi_tokens[i])

print(re_tokenized)
print(preprocess(ru_dataset[example_num], model_tokenizer))
print(preprocess(ru_dataset[example_num], enru_tokenizer))
print(preprocess(ru_dataset[example_num], enru_mtokenizer))

Sample: Иоанн Возлюбленный Учение о правильном использовании Слов, 4 августа 2010 г.
['▁Ио', 'ан', 'н', '▁Воз', 'люблен', 'ный', '▁У', 'чение', '▁', 'о', '▁правильн', 'ом', '▁использовани', 'и', '▁Слов', ',', '▁4', '▁август', 'а', '▁2010', '▁г', '.']
['▁Иоанн', '▁Возлюбленны', 'й', '▁Учени', 'е', '▁', 'о', '▁', 'правильном', '▁использовании', '▁Слов', ',', '▁4', '▁августа', '▁2010', '▁г.']
['ÐĺÐ¾Ð°Ð½Ð½', '▁ÐĴÐ¾Ð·Ð»ÑİÐ±Ð»ÐµÐ½Ð½ÑĭÐ¹', '▁Ð£ÑĩÐµÐ½Ð¸Ðµ', '▁Ð¾', '▁Ð¿ÑĢÐ°Ð²Ð¸Ð»ÑĮÐ½Ð¾Ð¼', '▁Ð¸ÑģÐ¿Ð¾Ð»ÑĮÐ·Ð¾Ð²Ð°Ð½Ð¸Ð¸', '▁Ð¡Ð»Ð¾Ð²', ',', '▁4', '▁Ð°Ð²Ð³ÑĥÑģÑĤÐ°', '▁2010', '▁Ð³', '.']
['▁Ио', 'ан', 'н', '▁Воз', 'люблен', 'ны', 'й', '▁У', 'чени', 'е', '▁', 'о', '▁правильн', 'ом', '▁использовани', 'и', '▁Слов', ',', '▁4', '▁август', 'а', '▁2010', '▁г', '.']
{'input_ids': [37194, 5413, 288, 20567, 267, 4040, 287, 1071, 28387, 259, 104683, 351, 287, 4645, 2225, 304, 15758, 261, 419, 3155, 1068, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [152395

In [139]:
print(model_tokenizer.tokenize(ru_dataset[example_num]["1"], max_length=128, truncation=True))

['▁Как', '▁правил', 'о', ',', '▁', 'наши', '▁', 'соо', 'течественн', 'ики', '▁хот', 'ят', '▁у', 'слыш', 'ать', '▁', 'еще', '▁одно', '▁мне', 'ние', '▁', 'авторитет', 'ного', '▁врач', 'а', ',', '▁пере', 'под', 'тверди', 'ть', '▁диагно', 'з', ':', '▁«', 'К', 'стати', ',', '▁в', '▁Германи', 'и', '▁за', 'част', 'ую', '▁с', 'мяг', 'чают', '▁диагно', 'з', ',', '▁предлага', 'ют', '▁мало', 'ин', 'ваз', 'ивное', '▁', 'лечение', '.']


In [11]:
def retokenize(tokens, new_tokenizer, new_tokenizer_vocab):
    re_tokenized = []
    for i in range(len(tokens)):
        if tokens[i] not in new_tokenizer_vocab:
            #print(tokens[i])
            new_toks = new_tokenizer.tokenize(tokens[i], add_special_tokens=False)
            #print(def_de_tok, de_tokens[i])
            if len(new_toks) > 0:
                if tokens[i][0] != '▁' and new_toks[0] == '▁':
                    new_toks = new_toks[1:]
                elif new_toks[0][0] == '▁' and len(re_tokenized) > 0 and re_tokenized[-1] == '▁':
                    del re_tokenized[-1]
                re_tokenized += new_toks
        else:
            re_tokenized.append(tokens[i])
    re_tokenized = (new_tokenizer.convert_tokens_to_ids(re_tokenized) + [1])
    return re_tokenized


def lang_preprocess(sample):
    inputs = prefix + str(sample["0"])
    targets = str(sample["1"])
    model_inputs = {}
    model_inputs["input_ids"] = ende_tokenizer.tokenize(inputs, max_length=128, truncation=True)
    model_inputs["input_ids"] = retokenize(model_inputs["input_ids"], model_tokenizer, set(model_tokenizer.get_vocab().keys()))
    labels = model_tokenizer.tokenize(targets, max_length=128, truncation=True)
    labels = retokenize(labels, model_tokenizer, model_tokenizer_vocab)
    model_inputs["attention_mask"] = ([1] * len(model_inputs["input_ids"]))
    model_inputs["labels"] = labels
    #print(model_inputs)
    return model_inputs

In [14]:
example_num = 7
print(preprocess(de_dataset[example_num], model_tokenizer))
print(lang_preprocess(de_dataset[example_num]))

{'input_ids': [37194, 5413, 288, 20567, 267, 642, 262, 75443, 287, 1546, 3415, 304, 26594, 259, 108880, 261, 287, 259, 57611, 2178, 259, 53567, 2178, 259, 128868, 511, 19532, 259, 73433, 263, 15848, 287, 259, 54148, 23317, 259, 4964, 259, 159713, 305, 10385, 332, 259, 4609, 151738, 36732, 14679, 2178, 3399, 16185, 5233, 5927, 3826, 46213, 21511, 345, 514, 381, 403, 14181, 288, 259, 38195, 26594, 484, 264, 204941, 265, 25997, 305, 10239, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [259, 22496, 259, 14416, 21638, 262, 6271, 286, 68041, 44050, 270, 309, 259, 57611, 2178, 259, 53567, 2178, 259, 106177, 511, 489, 264, 220805, 68041, 66934, 259, 45200, 272, 398, 42951, 9933, 259, 112444, 795, 259, 217071, 472, 58147, 749, 36732, 14679, 2178, 3399, 90849, 956, 67693, 119787, 3399, 2